In [ ]:
using DataFrames
using DICOM
using Juliana

# Config

In [ ]:
data_dir = "/data/user/bellotti_r/data_juliana_application"
patient_ID = "patient_01"
original_dose_file = "/data/user/bellotti_r/differentiable-planning/output/prof_weber_patient/patient_01/result_dose.dat"
plan_file = "/data/user/bellotti_r/differentiable-planning/output/prof_weber_patient/patient_01/result_plan.json"

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.9.jar";

In [ ]:
output_dir = "../output/test"
mkpath(output_dir)

# Load data

In [ ]:
ct_path, patient_data = Juliana.load_patient_data(data_dir, patient_ID);

In [ ]:
target_name, target_dose = Juliana.hottest_target(patient_data.prescriptions);

In [ ]:
plan = Juliana.FionaStandalone.read_plan_file(plan_file);

In [ ]:
spot_weights = vcat([DataFrame(f.spots).weight for f in plan.fields]...);

In [ ]:
optimisation_mask, optimisation_points, optimisation_point_indices = Juliana.get_optimisation_points_from_prescription(
    patient_data.ct.grid,
    patient_data.prescriptions,
    patient_data.structures,
    checkerboard_skip_n=4,
)

optimisation_grid = Juliana.get_optimisation_grid(
    optimisation_points,
    patient_data.ct.grid,
)

# Calculate the dose on the adaptive grid

In [ ]:
highres_grid = optimisation_grid

In [ ]:
dose = Juliana.FionaStandalone.calculate_dose_adaptive(
    plan,
    ct_path,
    patient_data.ct.grid,
    output_dir,
    fiona_standalone_bin_path,
    fiona_jar_path,
    optimisation_grid,
    highres_grid,
    low_res=0.35,
);

# Export to DICOM

In [ ]:
original_dose = Juliana.load_dose_dat_file(original_dose_file)

In [ ]:
dose = Juliana.ScalarGrid(
    dose.data,
    Juliana.read_dicom_dose("/data/user/bellotti_r/data_juliana_application/DICOM/patient_01/RD.dcm").grid,
)

dose_dcm = Juliana.dose_to_dicom(
    dose,
    patient_ID,
    patient_ID,
    "",
    "",
    "",
)
DICOM.dcm_write("$(output_dir)/RD_highres.dcm", dose_dcm)